# TBL Preguntas CSV Reader - Robust Parser ✅

This notebook reads the `tbl_preguntas.csv` file which has **malformed CSV structure** where the 'respuesta' column contains:
- Unescaped newlines within quoted fields
- CSV-escaped quotes (doubled quotes: `""`)
- Very long text content spanning many lines

## The Problem
- Standard CSV parsers fail because fields like `respuesta` have quotes that start but contain literal newlines
- Records like ID **588, 595, 5475, 5181** span multiple lines and break normal parsing
- Some fields contain escaped quotes using the CSV convention of doubling them

## The Solution
We use a **character-by-character state machine parser** that:
1. Tracks whether we're inside or outside quoted fields
2. Properly handles CSV-escaped quotes (`""` → `"`)
3. Accumulates multiline content within quotes
4. Validates complete records (18 columns expected)

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Custom CSV Parser Implementation

In [2]:
def parse_csv_with_multiline_fields(file_path):
    """
    Parse CSV file that has newlines within quoted fields.
    Handles CSV-escaped quotes (doubled quotes: "").
    
    Uses a character-by-character state machine approach:
    - Tracks quote state to know when we're inside/outside quoted fields
    - Handles escaped quotes ("" becomes ")
    - Accumulates lines until complete record is found
    """
    records = []
    current_record = []
    current_field = []
    in_quotes = False
    header = None
    prev_char = ''

    with open(file_path, 'r', encoding='utf-8') as f:
        char_iter = iter(lambda: f.read(1), '')

        for char in char_iter:
            if char == '"':
                # Check if this is an escaped quote (doubled: "")
                if prev_char == '"' and in_quotes:
                    # This is an escaped quote - add one quote to field
                    current_field.append('"')
                    prev_char = ''  # Reset to avoid triple-quote issues
                    continue
                else:
                    # Toggle quote state
                    in_quotes = not in_quotes
                    prev_char = char
                    continue

            if not in_quotes:
                if char == ',':
                    # End of field
                    current_record.append(''.join(current_field))
                    current_field = []
                elif char == '\n':
                    # End of record
                    if current_field or current_record:
                        current_record.append(''.join(current_field))
                        current_field = []

                        if header is None:
                            header = current_record
                        else:
                            if len(current_record) == len(header):
                                records.append(current_record)
                        current_record = []
                else:
                    current_field.append(char)
            else:
                # Inside quotes - add everything including newlines
                current_field.append(char)

            prev_char = char

        # Handle last field/record
        if current_field or current_record:
            current_record.append(''.join(current_field))
            if len(current_record) == len(header):
                records.append(current_record)

    df = pd.DataFrame(records, columns=header)
    return df

## Load the CSV File

In [3]:
print("Loading tbl_preguntas.csv with custom parser...")
print("This may take a moment...\n")

df = parse_csv_with_multiline_fields('tbl_preguntas.csv')

print(f"✅ Successfully loaded {len(df):,} records")
print(f"\nColumns ({len(df.columns)}): {list(df.columns)}")

Loading tbl_preguntas.csv with custom parser...
This may take a moment...

✅ Successfully loaded 1,489 records

Columns (18): ['idtbl_pregunta', 'titulo', 'respuesta', 'id_usuario_creador', 'fecha_creacion', 'id_usuario_ultima_modificacion', 'fecha_ultima_modificacion', 'id_producto', 'id_estado', 'fecha_ult_actualizacion_keywords', 'keywords_rag', 'Vigencia', 'estado_carga', 'dias_vigencia', 'justificacion_garante', 'justificacion_curador', 'publicado_workplace', 'fecha_publicacion']


In [4]:
# Convert numeric columns to proper types
numeric_cols = ['idtbl_pregunta', 'id_usuario_creador', 'id_usuario_ultima_modificacion',
                'id_producto', 'id_estado', 'estado_carga', 'dias_vigencia']

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print("✅ Converted numeric columns")

✅ Converted numeric columns


## ✅ Verify Problematic Records

Let's check the specific IDs you mentioned that had parsing issues: **588, 595, 5475, 5181**

In [5]:
# Check specific problematic IDs
problematic_ids = [588, 595, 5475, 5181]

print("=" * 100)
print("VERIFICATION OF PROBLEMATIC RECORDS")
print("=" * 100)

found_count = 0
for pid in problematic_ids:
    matching_rows = df[df['idtbl_pregunta'] == pid]
    if len(matching_rows) > 0:
        found_count += 1
        print(f"\n✅ ID {pid} FOUND")
        print("-" * 100)
        row = matching_rows.iloc[0]
        print(f"Titulo: {row['titulo']}")
        print(f"\nRespuesta stats:")
        resp_text = str(row['respuesta'])
        print(f"  - Length: {len(resp_text):,} characters")
        print(f"  - Newlines: {resp_text.count(chr(10))}")
        print(f"  - Contains quotes: {'Yes' if '"' in resp_text else 'No'}")
        print(f"\nFirst 300 chars of respuesta:")
        print(resp_text[:300])
        print("...")
    else:
        print(f"\n❌ ID {pid} NOT FOUND")

print("\n" + "=" * 100)
if found_count == len(problematic_ids):
    print(f"🎉 SUCCESS! All {found_count}/{len(problematic_ids)} problematic IDs loaded correctly!")
else:
    print(f"⚠️  Found {found_count}/{len(problematic_ids)} IDs")
print("=" * 100)

VERIFICATION OF PROBLEMATIC RECORDS

✅ ID 588 FOUND
----------------------------------------------------------------------------------------------------
Titulo: ¿Qué aspectos debo tener en cuenta para efectuar un ajuste por concepto de: GMF 4X1000, BRP, Uso de RPD y Egreso 25?

Respuesta stats:
  - Length: 6,066 characters
  - Newlines: 62
  - Contains quotes: No

First 300 chars of respuesta:
GMF: Recordemos que todos los ajustes por GMF (4X1000) deben contar con el VoBo del Dpto de impuestos.Si el ajuste solicitado es autorizado la instrucción del proceso, es decir, los conceptos y la forma en que se debe efectuar la transacción de ajuste será dada por el dpto de impuestos exclusivament
...

✅ ID 595 FOUND
----------------------------------------------------------------------------------------------------
Titulo: ¿Qué es mi información confidencial?

Respuesta stats:
  - Length: 2,949 characters
  - Newlines: 32
  - Contains quotes: Yes

First 300 chars of respuesta:
"Mi Información 

## Full Record Inspection

In [6]:
# Show one complete problematic record (ID 588)
if len(df[df['idtbl_pregunta'] == 588]) > 0:
    print("\n" + "=" * 100)
    print("FULL RECORD DETAILS FOR ID 588")
    print("=" * 100)
    record = df[df['idtbl_pregunta'] == 588].iloc[0]
    for col in df.columns:
        value = record[col]
        if col == 'respuesta':
            print(f"\n{col}:")
            print(f"  Length: {len(str(value)):,} chars")
            print(f"  Newlines: {str(value).count(chr(10))}")
            print(f"  Preview (first 500 chars):\n{str(value)[:500]}...")
        else:
            print(f"{col}: {value}")


FULL RECORD DETAILS FOR ID 588
idtbl_pregunta: 588.0
titulo: ¿Qué aspectos debo tener en cuenta para efectuar un ajuste por concepto de: GMF 4X1000, BRP, Uso de RPD y Egreso 25?

respuesta:
  Length: 6,066 chars
  Newlines: 62
  Preview (first 500 chars):
GMF: Recordemos que todos los ajustes por GMF (4X1000) deben contar con el VoBo del Dpto de impuestos.Si el ajuste solicitado es autorizado la instrucción del proceso, es decir, los conceptos y la forma en que se debe efectuar la transacción de ajuste será dada por el dpto de impuestos exclusivamente.

BRP: Los conceptos habilitados para el registro correcto de las operaciones de BRP son:

1. Devolución cuota inicial BRP: Devolución de la cuota del BRP al cliente por cualquier concepto, Referenc...
id_usuario_creador: 88.0
fecha_creacion: 2020-01-24 00:00:00
id_usuario_ultima_modificacion: 8036.0
fecha_ultima_modificacion: 2025-10-03 11:45:40
id_producto: 182.0
id_estado: 6.0
fecha_ult_actualizacion_keywords: 2025-04-22 19:00:00
keyw

## Data Overview

In [ ]:
# Basic info
print("DataFrame Info:")
df.info()

In [ ]:
# First few rows (with truncated respuesta for readability)
display_df = df.head(10).copy()
display_df['respuesta'] = display_df['respuesta'].str[:80] + '...'
display_df

## Data Quality Checks

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

In [ ]:
# Check for duplicate IDs
duplicates = df[df.duplicated(subset=['idtbl_pregunta'], keep=False)]
if len(duplicates) > 0:
    print(f"⚠️  Found {len(duplicates)} duplicate records:")
    print(duplicates[['idtbl_pregunta', 'titulo']].sort_values('idtbl_pregunta'))
else:
    print("✅ No duplicate IDs found")

In [ ]:
# Analyze multiline content in respuesta
df['_newline_count'] = df['respuesta'].astype(str).apply(lambda x: x.count('\n'))
df['_char_count'] = df['respuesta'].astype(str).apply(len)

print("\nMultiline content analysis:")
print(f"Records with 0 newlines: {len(df[df['_newline_count'] == 0]):,}")
print(f"Records with 1-10 newlines: {len(df[(df['_newline_count'] > 0) & (df['_newline_count'] <= 10)]):,}")
print(f"Records with 11-50 newlines: {len(df[(df['_newline_count'] > 10) & (df['_newline_count'] <= 50)]):,}")
print(f"Records with 50+ newlines: {len(df[df['_newline_count'] > 50]):,}")

print(f"\n\nTop 10 records with most newlines:")
top_multiline = df.nlargest(10, '_newline_count')[['idtbl_pregunta', 'titulo', '_newline_count', '_char_count']]
print(top_multiline.to_string())

## Summary Statistics

In [ ]:
print("\n" + "=" * 100)
print("DATASET SUMMARY")
print("=" * 100)
print(f"Total records: {len(df):,}")
print(f"\nDate range:")
print(f"  Creation: {df['fecha_creacion'].min()} to {df['fecha_creacion'].max()}")
print(f"  Last modified: {df['fecha_ultima_modificacion'].min()} to {df['fecha_ultima_modificacion'].max()}")
print(f"\nUnique products: {df['id_producto'].nunique()}")
print(f"Unique states: {df['id_estado'].nunique()}")

print(f"\nRecords by estado_carga:")
print(df['estado_carga'].value_counts())

print(f"\nRecords by id_estado:")
print(df['id_estado'].value_counts())

## Clean Up Helper Columns

In [ ]:
# Remove temporary analysis columns
df_clean = df.drop(['_newline_count', '_char_count'], axis=1, errors='ignore')
print(f"✅ Cleaned dataframe has {len(df_clean.columns)} columns")

## Export Options

### Option 1: Export to Parquet (Recommended)

Parquet format handles multiline text perfectly and is much more efficient than CSV:

In [ ]:
# Uncomment to save
# df_clean.to_parquet('tbl_preguntas.parquet', index=False, engine='pyarrow')
# print("✅ Data saved to tbl_preguntas.parquet")
# print("\nTo read back: df = pd.read_parquet('tbl_preguntas.parquet')")

### Option 2: Export to properly formatted CSV

In [ ]:
# Uncomment to save
# import csv
# df_clean.to_csv('tbl_preguntas_cleaned.csv', index=False, encoding='utf-8', 
#                 quoting=csv.QUOTE_ALL, escapechar=None, doublequote=True)
# print("✅ Cleaned data exported to tbl_preguntas_cleaned.csv")

### Option 3: Export to Excel

In [ ]:
# Uncomment to save
# df_clean.to_excel('tbl_preguntas.xlsx', index=False, engine='openpyxl')
# print("✅ Data saved to tbl_preguntas.xlsx")

## Working with the Data

Now you can use `df` or `df_clean` for your analysis. All problematic records are successfully loaded!

In [ ]:
# Example: Search for questions containing specific keywords
keyword = "ajuste"  # Change this to search for different keywords
results = df_clean[df_clean['titulo'].str.contains(keyword, case=False, na=False)]
print(f"Found {len(results)} questions containing '{keyword}':")
results[['idtbl_pregunta', 'titulo']].head(10)